# Keyword Frequencies

Compute the frequencies of keywords over time

In [12]:
import os
import pandas as pd
import re
from IPython.display import clear_output

In [30]:
DATA_DIR = "../data/text/"
DATE_COUNTS_FILE = "../data/date-counts.csv"
ID_STR = "id_str"
TEXT = "text"

In [11]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [22]:
def search_text(query, date_pattern, counts):
    file_names = sorted(os.listdir(DATA_DIR))
    counts_new = {}
    for file_name in file_names:
        if re.search(date_pattern, file_name):
            date = file_name[0:8]
            if date not in counts:
                if date not in counts_new: 
                    counts_new[date] = 0
                    squeal(f"{date} {query}")
                df = pd.read_csv(DATA_DIR + file_name, index_col=ID_STR)
                counts_new[date] += len(df[df[TEXT].str.contains(r'\b' + query + r'\b', flags=re.IGNORECASE)])
    counts.update(counts_new)
    return 

In [28]:
def get_counts(queries, date_pattern):
    counts = {}
    for query in queries:
        counts[query] = {}
        search_text(query, date_pattern, counts[query])
    return counts

In [70]:
def add_frequencies(counts):
    date_counts = pd.read_csv(DATE_COUNTS_FILE, index_col="date")
    counts_extra = {}
    counts_extra["aantal tweets"] = {}
    for query in counts:
        frequency_query = f"frequentie {query}"
        counts_extra[frequency_query] = {}
        for date in counts[query]:
            if date not in counts_extra["aantal tweets"]:
                counts_extra["aantal tweets"][date] = date_counts.loc[int(date)]["count"]
            counts_extra[frequency_query][date] = counts[query][date] / counts_extra["aantal tweets"][date]
    counts.update(counts_extra)

In [76]:
QUERIES = [ "populisme", "populismes", "populist", "populisten", "populistisch", "populistische" ]

counts = get_counts(QUERIES, "^2019")

20191231 populistische


In [79]:
counts_copy = dict(counts)
add_frequencies(counts_copy)

In [80]:
pd.DataFrame(counts_copy).to_csv("keyword_frequencies.csv", index_label="datum")